In [1]:
import json
import re
import os
import gzip
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial


spectra_map = {0: 'M', 1: 'K', 2: 'G', 3: 'F', 4: 'A', 5: 'B', 6: 'O', 7: 'X'}
singularity_flags = {
    'TidalLocked': 1,
    'TidalLocked2': 2,
    'TidalLocked4': 4,
    'LaySide': 8,
    'ClockwiseRotate': 16,
    'MultipleSatellites': 32,
}
star_type_map = {
    0: 'Main Sequence',
    1: 'Giant',
    2: 'White Dwarf',
    3: 'Neutron Star',
    4: 'Black Hole',
}
vein_types = [
    'Iron', 'Copper', 'Silicium', 'Titanium', 'Stone', 'Coal', 'Oil', 'Fireice', 
    'Diamond', 'Fractal', 'Crysrub', 'Grat', 'Bamboo', 'Mag',
]
gas_types = [
    'Hydrogen gas', 'Deuterium gas', 'Fire ice gas',
]
previous_runs = [ 
    84459701, 16015752, 22778246, 26027001, 57786559, 51279269, 94520636, 
    32680784, 88834638, 22161804, 87759803, 76768927, 18238299,
]

In [2]:
export_path = '/mnt/c/' + r'\Users\Drake\Documents\Dyson Sphere Program\dsp_seedexporter'.replace('\\', '/')
# export_path = '/run/user/1000/gvfs/smb-share:server=chonyi.local,share=dsp_seedexporter/'
seed_re = re.compile(r'seed_(?P<seed>\d{8}).json.gz')

In [3]:
files = sorted(filter(seed_re.match, os.listdir(export_path)))
seeds = list(map(lambda x: seed_re.match(x).group('seed'), files))

In [4]:
def process_universe(func, file):
    try:
        with gzip.GzipFile(os.path.join(export_path, file), mode='r') as fd:
            universe = json.load(fd)
    except EOFError as e:
        raise ValueError('Bad file: ' + file)
    return func(universe)

def passthrough(x):
    return x

subset = ['seed_18238299.json.gz', 'seed_94520636.json.gz', 'seed_94520637.json.gz']
with mp.Pool(mp.cpu_count() - 1) as pool:
    universes = pool.map(partial(process_universe, passthrough), subset, chunksize=100)
universe = universes[0]

In [5]:
def check_files(file):
    try:
        with gzip.GzipFile(os.path.join(export_path, file), mode='r') as fd:
            universe = json.load(fd)
    except EOFError as e:
        return file

with mp.Pool(mp.cpu_count() - 1) as pool:
    bad_files = [x for x in pool.imap_unordered(check_files, files, chunksize=100) if x is not None]
if bad_files:
    raise IOError(bad_files)

In [6]:
for bad_file in bad_files:
    os.remove(os.path.join(export_path, bad_file))

In [7]:
def distance(star):
    return sum(np.fromiter(star['position'].values(), dtype='float') ** 2) ** 0.5

distance_df = pd.DataFrame.from_records(
    data=[(star_name, distance(star)) for star_name, star in universe['star'].items()], 
    columns=['star_name', 'abs. dist.']
).sort_values('abs. dist.')
distance_df

,star_name,abs. dist.
0,Procyon,0.000000
1,Tramontana,2.960353
16,Zeta Arietis,4.005012
17,27 Ophiuchii,4.611032
4,Lambda Sagittae,5.189216
...,...,...
40,ζ Chamaeleontis,27.360977
45,Cymbae,30.245616
51,Theta Puppis,33.166251
60,Arided,36.442384


In [8]:
home_star = distance_df.iloc[0]
assert home_star['abs. dist.'] == 0.
home_system = universe['star'][home_star['star_name']]

In [9]:
def map_strings(universe):
    planet_types = []
    star_types = []
    singularities = []
    planet_count = 0
    for star in universe['star'].values():
        try:
            star_types.append((star['type'], star['typeString']))
        except KeyError:
            pass
        for planet in star['planet'].values():
            try:
                singularities.append((planet['singularity'], planet['singularityString']))
            except KeyError:
                pass
            try:
                planet_types.append((planet['type'], planet['typeString']))
            except KeyError:
                pass
            planet_count += 1
    return (
        np.unique(planet_types, axis=0), 
        np.unique(singularities, axis=0), 
        np.unique(star_types, axis=0), 
        planet_count
    )


with mp.Pool(mp.cpu_count() - 1) as pool:
    planet_types, singularities, star_types, planet_count = zip(*pool.imap_unordered(partial(process_universe, map_strings), files, chunksize=100))
planet_types = np.unique(np.concatenate([x for x in planet_types if len(x) > 0]), axis=0)
singularities = np.unique(np.concatenate([x for x in singularities if len(x) > 0]), axis=0)
star_types = np.unique(np.concatenate([x for x in star_types if len(x) > 0]), axis=0)


In [10]:
display(pd.DataFrame.from_records(planet_types, columns=['code', 'type']))
display(pd.DataFrame.from_records(star_types, columns=['code', 'type']))

,code,type
0,1,Lava
1,1,Volcanic Ash
2,2,Mediterranean
3,2,Oceanic Jungle
4,2,Prairie
5,2,Red Stone
6,2,Sakura Ocean
7,2,Waterworld
8,3,Arid Desert
9,3,Ashen Gelisol


,code,type
0,0,A type star
1,0,B type star
2,0,F type star
3,0,G type star
4,0,K type star
5,0,M type star
6,0,O type star
7,1,Blue giant
8,1,Red giant
9,1,White giant


In [11]:
def scan_systems(universe):
    home_star_id = universe['meta']['birthStarId']
    home_planet_id = universe['meta']['birthPlanetId']
    seed = universe['meta']['seed']
    stars = []
    planets = []
    
    for star in universe['star'].values():
        max_dyson_rad = star['dysonRadius'] * 2

        star_dict = {
            'seed': seed,
            'star': star['name'],
            'homeworld': star['id'] == home_star_id,
            'spectrum': spectra_map[star['spectr']],
            'type': star_type_map[star['type']],
            'luminosity': star['luminosity'],
        }
        stars.append(star_dict)

        for planet in star['planet'].values():
            planet_dict = {
                'seed': seed,
                'star': star['name'],
                'planet': planet['name'],
                'home_system': star['id'] == home_star_id,
                'homeworld': planet['id'] == home_planet_id,
                'type': planet.get('typeString', None),
                'tidal_lock': (planet['singularity'] & singularity_flags['TidalLocked']) > 0,
                'in_sphere': (1.36 * planet['sunDistance']) <= max_dyson_rad,
            }
            if 'vein' in planet:
                planet_veins = planet['vein']
#                 planet_veins = {x: planet['vein'].get(x, 0) for x in resource_types}
            elif 'gas' in planet:
                planet_veins = {(k + ' gas'): v['gasSpeed'] for k,v in planet['gas'].items()}
            else:
                raise ValueError('Bad planet: {}'.format(planet['name']))
            planets.append({**planet_dict, **planet_veins})
    planet_df = pd.DataFrame.from_records(planets)    
    star_df = pd.DataFrame.from_records(stars)
    star_resources = planet_df.groupby('star')[vein_types + gas_types].sum()
    star_df = star_df.merge(star_resources, left_on='star', right_index=True, how='outer')
    return star_df, planet_df
star_df, planet_df = scan_systems(universe)
display(star_df.head())
display(planet_df.head())

,seed,star,homeworld,spectrum,type,luminosity,Iron,Copper,Silicium,Titanium,Stone,Coal,Oil,Fireice,Diamond,Fractal,Crysrub,Grat,Bamboo,Mag
0,18238299,Procyon,True,G,Main Sequence,0.978408,23539812.0,33240354.0,13467962.0,13893383.0,15915882.0,12433108.0,1330597.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,18238299,Tramontana,False,M,Main Sequence,0.583942,14053173.0,28310788.0,3529581.0,37695658.0,24063714.0,451813.0,0.0,5151677.0,0.0,878159.0,0.0,2664609.0,0.0,0.0
2,18238299,Gamma Vulpeculae,False,K,Main Sequence,0.823254,36914259.0,41160402.0,20979566.0,37358159.0,22529836.0,4269425.0,0.0,14143138.0,0.0,0.0,0.0,0.0,0.0,0.0
3,18238299,YedPrior,False,M,Main Sequence,0.497194,26414756.0,18726224.0,19573996.0,39492924.0,8379710.0,565422.0,0.0,12073814.0,0.0,1734430.0,0.0,0.0,0.0,0.0
4,18238299,Lambda Sagittae,False,K,Main Sequence,0.817663,9531719.0,1975948.0,4960219.0,31078570.0,46256463.0,159363.0,0.0,16256882.0,0.0,0.0,0.0,958814.0,0.0,0.0


,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,Iron,Copper,...,Hydrogen gas,Oil,Fireice,Fractal,Grat,Bamboo,Diamond,Crysrub,Deuterium gas,Mag
0,18238299,Procyon,Procyon I,True,False,Lava,False,True,19746067.0,22095603.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18238299,Procyon,Procyon II,True,False,Gobi,False,False,892945.0,9786748.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18238299,Procyon,Procyon III,True,False,Ice Giant,False,False,NaN,NaN,...,0.2508723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18238299,Procyon,Procyon IV,True,True,Mediterranean,False,False,2900800.0,1358003.0,...,NaN,1330597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238299,Tramontana,Tramontana I,False,False,Arid Desert,False,True,1682756.0,10781232.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
with mp.Pool(mp.cpu_count() - 1) as pool:
    star_dfs, planet_dfs = zip(*pool.imap_unordered(partial(process_universe, scan_systems), files, chunksize=100))
star_df = pd.concat(star_dfs)
planet_df = pd.concat(planet_dfs)

In [13]:
planet_df.query('(tidal_lock == True) & (home_system == True)')

,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,Iron,Copper,...,Deuterium gas,Oil,Crysrub,Bamboo,Fire ice gas,Fireice,Fractal,Diamond,Grat,Mag
0,27395841,Delta Hydrae,Delta Hydrae I,True,False,Lava,True,True,18701597.0,20183690.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,28377642,Clava,Clava I,True,False,Rocky Salt Lake,True,True,928383.0,5538272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,29399682,Gacrux,Gacrux I,True,False,Arid Desert,True,False,832305.0,7694018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36049318,Merope,Merope II,True,False,None,True,False,20497680.0,20048346.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,36466079,Agena,Agena I,True,False,None,True,False,476903.0,8449588.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,93493528,AsellusPrimus,AsellusPrimus I,True,False,None,True,True,20275893.0,23628545.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,94520636,ι Coronae Borealis,ι Coronae Borealis II,True,False,None,True,False,810903.0,6122990.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,94520642,Sheratan,Sheratan I,True,False,Lava,True,True,20520276.0,19582585.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,94523164,Iota Delphini,Iota Delphini I,True,False,Gobi,True,True,1280194.0,10085521.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
